Quick model to establish a pipeline and simple benchmark

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression, RidgeClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from nltk import word_tokenize, sent_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm import tqdm, tqdm_pandas
import string
import re
from gensim.models import Word2Vec, Phrases
from gensim.utils import to_unicode
from gensim.models.phrases import Phraser
import logging

%matplotlib inline
pd.set_option('display.max_columns', None)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

c:\users\klein535\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
sample_submission_data = pd.read_csv('data/sample_submission.csv')

In [3]:
train_data.head()
train_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

### Define useful helper functions

Function for cleaning and tokenizing:

In [4]:
stopset = set(stopwords.words('english')) - set(('over', 'under', 'below', 'more', 'most', 'no', 'not', 'only', 
                                                 'such', 'few', 'so', 'too', 'very', 'just', 'any', 'once'))
snow = SnowballStemmer('english')
WNlemma = WordNetLemmatizer()

def clean_text(x, normalization='stemming', for_word2vec=False):
    # split text
    words = word_tokenize(x)
    
    # remove punctuation and numbers
    words = [to_unicode(word.strip(string.punctuation).lower()) for word in words if word not in string.punctuation and not bool(re.search(r'\d', word))]
    
    if normalization == 'stemming':
        words = [snow.stem(t) for t in words] # stemming
    elif normalization == 'lemmatization':
        words = [WNlemma.lemmatize(t) for t in words] # lemmatize words (advanced stemming)
    else:
        return words
    
    if for_word2vec:
        return words
    else:
        # join list into string (needed for CountVectorizer) and remove underscore characters from words
        joined_words = ' '.join(words).replace('_', '')
    
    return joined_words

### Prepare corpus for Word2Vec training

In [5]:
# get corpus by concatenating train and test comments
corpus = train_data['comment_text'].copy().append(test_data['comment_text'].copy())

# join into one large string
corpus = '. '.join(corpus)
corpus = corpus.replace('\n', '. ')

# split into sentences
corpus = pd.Series(sent_tokenize(corpus))

In [6]:
tqdm.pandas(tqdm()) # for tracking progress
corpus = corpus.progress_apply(lambda x: clean_text(x, normalization='lemmatization', for_word2vec=True))

0it [00:00, ?it/s]
  1%|▉                                                                       | 25419/1975667 [00:08<08:45, 3712.08it/s]


  3%|█▉                                                                      | 52431/1975667 [00:15<10:47, 2971.07it/s]


  4%|██▊                                                                     | 78827/1975667 [00:23<08:41, 3634.51it/s]


  5%|███▊                                                                   | 104944/1975667 [00:30<08:29, 3669.63it/s]


  7%|████▋                                                                  | 131203/1975667 [00:37<08:00, 3838.29it/s]


...

 97%|███████████████████████████████████████████████████████████████████▋  | 1910383/1975667 [10:03<00:25, 2551.21it/s]


 98%|████████████████████████████████████████████████████████████████████▍ | 1930423/1975667 [10:11<00:15, 2843.64it/s]


 99%|█████████████████████████████████████████████████████████████████████▏| 1951811/1975667 [10:18<00:08, 2769.30it/s]


100%|█████████████████████████████████████████████████████████████████████▉| 1973046/1975667 [10:25<00:00, 2755.64it/s]


100%|██████████████████████████████████████████████████████████████████████| 1975667/1975667 [10:26<00:00, 3151.32it/s]

In [7]:
# residual cleaning: remove empty elements from list
corpus = list(filter(None, corpus))
corpus[120000:122000]

[['here', 'the', 'liknk', 'from', 'before'],
 [''],
 ['you',
  'wrote',
  'i',
  'myself',
  'am',
  'christian',
  'but',
  'i',
  'will',
  'not',
  'allow',
  'myself',
  'the',
  'shortcut',
  'of',
  'rationalising',
  'that',
  'rape',
  'turned',
  'an',
  'otherwise',
  'straight',
  'lonnie',
  'frisbee',
  'into',
  'a',
  '',
  '',
  'gay',
  'sinner',
  '',
  ''],
 ['well',
  '',
  'i',
  'myself',
  'am',
  'christian',
  'but',
  'i',
  'will',
  'not',
  'allow',
  'myself',
  'lying',
  'gay',
  'acitivists',
  'to',
  'pretend',
  'that',
  'rape',
  'doe',
  "not'turned",
  'an',
  'otherwise',
  'straight',
  'lonnie',
  'frisbee',
  'into',
  'a',
  '',
  '',
  'gay',
  'sinner',
  '',
  ''],
 ['i',
  'think',
  'the',
  'only',
  'reason',
  'why',
  'you',
  'lying',
  'gay',
  'activist',
  'wo',
  "n't",
  'acknowledge',
  'this',
  'phenomenon',
  'is',
  'at',
  'it',
  's',
  'very',
  'best',
  'it',
  'cut',
  'into',
  'your',
  'argument',
  'that',
  'pe

### Train and store Word2Vec models

In [8]:
# 5 epochs
model = Word2Vec(corpus, min_count=5, size=100, workers=4, iter=5)
model.save('./models/5_epoch_word2vec_model')
print('5 epoch model trained and saved.')

# 15 epochs
model = Word2Vec(corpus, min_count=5, size=100, workers=4, iter=15)
model.save('./models/15_epoch_word2vec_model')
print('15 epoch model trained and saved.')

# 50 epochs
model = Word2Vec(corpus, min_count=5, size=100, workers=4, iter=50)
model.save('./models/50_epoch_word2vec_model')
print('50 epoch model trained and saved.')

2018-02-05 13:28:28,251 : INFO : collecting all words and their counts
2018-02-05 13:28:28,253 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-05 13:28:28,312 : INFO : PROGRESS: at sentence #10000, processed 137791 words, keeping 13073 word types
2018-02-05 13:28:28,363 : INFO : PROGRESS: at sentence #20000, processed 274707 words, keeping 19530 word types
2018-02-05 13:28:28,407 : INFO : PROGRESS: at sentence #30000, processed 404356 words, keeping 24367 word types
2018-02-05 13:28:28,456 : INFO : PROGRESS: at sentence #40000, processed 540595 words, keeping 29137 word types
2018-02-05 13:28:28,506 : INFO : PROGRESS: at sentence #50000, processed 674837 words, keeping 33406 word types
2018-02-05 13:28:28,553 : INFO : PROGRESS: at sentence #60000, processed 811450 words, keeping 37494 word types
2018-02-05 13:28:28,605 : INFO : PROGRESS: at sentence #70000, processed 943368 words, keeping 40965 word types
2018-02-05 13:28:28,658 : INFO : PROGRESS: at 

2018-02-05 13:28:32,008 : INFO : PROGRESS: at sentence #710000, processed 9582530 words, keeping 178767 word types
2018-02-05 13:28:32,073 : INFO : PROGRESS: at sentence #720000, processed 9720191 words, keeping 180568 word types
2018-02-05 13:28:32,124 : INFO : PROGRESS: at sentence #730000, processed 9852341 words, keeping 182234 word types
2018-02-05 13:28:32,171 : INFO : PROGRESS: at sentence #740000, processed 9988181 words, keeping 183921 word types
2018-02-05 13:28:32,215 : INFO : PROGRESS: at sentence #750000, processed 10121033 words, keeping 185446 word types
2018-02-05 13:28:32,263 : INFO : PROGRESS: at sentence #760000, processed 10255373 words, keeping 187046 word types
2018-02-05 13:28:32,311 : INFO : PROGRESS: at sentence #770000, processed 10392135 words, keeping 188796 word types
2018-02-05 13:28:32,368 : INFO : PROGRESS: at sentence #780000, processed 10530343 words, keeping 190403 word types
2018-02-05 13:28:32,421 : INFO : PROGRESS: at sentence #790000, processed 10

2018-02-05 13:28:35,615 : INFO : PROGRESS: at sentence #1410000, processed 18932348 words, keeping 345810 word types
2018-02-05 13:28:35,665 : INFO : PROGRESS: at sentence #1420000, processed 19066823 words, keeping 348269 word types
2018-02-05 13:28:35,734 : INFO : PROGRESS: at sentence #1430000, processed 19201567 words, keeping 350080 word types
2018-02-05 13:28:35,779 : INFO : PROGRESS: at sentence #1440000, processed 19330248 words, keeping 352105 word types
2018-02-05 13:28:35,825 : INFO : PROGRESS: at sentence #1450000, processed 19468809 words, keeping 354296 word types
2018-02-05 13:28:35,871 : INFO : PROGRESS: at sentence #1460000, processed 19602146 words, keeping 356065 word types
2018-02-05 13:28:35,919 : INFO : PROGRESS: at sentence #1470000, processed 19730119 words, keeping 358035 word types
2018-02-05 13:28:35,966 : INFO : PROGRESS: at sentence #1480000, processed 19862711 words, keeping 360149 word types
2018-02-05 13:28:36,013 : INFO : PROGRESS: at sentence #1490000,

2018-02-05 13:29:23,653 : INFO : EPOCH 2 - PROGRESS: at 86.43% examples, 605906 words/s, in_qsize 8, out_qsize 0
2018-02-05 13:29:24,657 : INFO : EPOCH 2 - PROGRESS: at 89.86% examples, 601168 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:29:25,660 : INFO : EPOCH 2 - PROGRESS: at 93.10% examples, 595853 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:29:26,677 : INFO : EPOCH 2 - PROGRESS: at 96.72% examples, 592692 words/s, in_qsize 8, out_qsize 0
2018-02-05 13:29:27,641 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-02-05 13:29:27,646 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-05 13:29:27,652 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-02-05 13:29:27,669 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-02-05 13:29:27,674 : INFO : EPOCH - 2 : training on 20928959 raw words (14909375 effective words) took 25.3s, 588769 effective words/s
2018-02-05 13:29:28,731 : INFO : EPOCH 3 - P

2018-02-05 13:30:25,906 : INFO : EPOCH 5 - PROGRESS: at 56.38% examples, 702340 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:30:26,908 : INFO : EPOCH 5 - PROGRESS: at 61.13% examples, 702514 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:30:27,927 : INFO : EPOCH 5 - PROGRESS: at 66.08% examples, 702470 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:30:28,930 : INFO : EPOCH 5 - PROGRESS: at 71.13% examples, 705391 words/s, in_qsize 8, out_qsize 1
2018-02-05 13:30:29,932 : INFO : EPOCH 5 - PROGRESS: at 76.00% examples, 706218 words/s, in_qsize 8, out_qsize 1
2018-02-05 13:30:30,945 : INFO : EPOCH 5 - PROGRESS: at 81.21% examples, 708957 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:30:31,959 : INFO : EPOCH 5 - PROGRESS: at 86.24% examples, 709863 words/s, in_qsize 5, out_qsize 2
2018-02-05 13:30:32,966 : INFO : EPOCH 5 - PROGRESS: at 91.17% examples, 710570 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:30:33,967 : INFO : EPOCH 5 - PROGRESS: at 96.12% examples, 711012 words/s, in_qsiz

...

2018-02-05 13:48:57,542 : INFO : EPOCH 39 - PROGRESS: at 49.91% examples, 828129 words/s, in_qsize 4, out_qsize 0
2018-02-05 13:48:58,548 : INFO : EPOCH 39 - PROGRESS: at 55.32% examples, 826087 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:48:59,550 : INFO : EPOCH 39 - PROGRESS: at 60.83% examples, 825384 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:49:00,576 : INFO : EPOCH 39 - PROGRESS: at 66.68% examples, 825795 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:49:01,577 : INFO : EPOCH 39 - PROGRESS: at 72.18% examples, 824784 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:49:02,575 : INFO : EPOCH 39 - PROGRESS: at 77.94% examples, 826176 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:49:03,599 : INFO : EPOCH 39 - PROGRESS: at 83.58% examples, 825935 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:49:04,614 : INFO : EPOCH 39 - PROGRESS: at 89.04% examples, 823031 words/s, in_qsize 7, out_qsize 1
2018-02-05 13:49:05,616 : INFO : EPOCH 39 - PROGRESS: at 94.61% examples, 822679 words/s

2018-02-05 13:49:57,629 : INFO : EPOCH 42 - PROGRESS: at 76.78% examples, 815083 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:49:58,644 : INFO : EPOCH 42 - PROGRESS: at 82.37% examples, 814660 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:49:59,644 : INFO : EPOCH 42 - PROGRESS: at 87.91% examples, 814490 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:50:00,646 : INFO : EPOCH 42 - PROGRESS: at 93.23% examples, 812511 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:50:01,647 : INFO : EPOCH 42 - PROGRESS: at 98.83% examples, 812859 words/s, in_qsize 8, out_qsize 0
2018-02-05 13:50:01,853 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-02-05 13:50:01,869 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-05 13:50:01,879 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-02-05 13:50:01,885 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-02-05 13:50:01,887 : INFO : EPOCH - 42 : training on 20928959

2018-02-05 13:50:57,180 : INFO : EPOCH 45 - PROGRESS: at 70.14% examples, 695178 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:50:58,181 : INFO : EPOCH 45 - PROGRESS: at 75.20% examples, 698360 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:50:59,195 : INFO : EPOCH 45 - PROGRESS: at 80.33% examples, 700686 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:51:00,196 : INFO : EPOCH 45 - PROGRESS: at 85.08% examples, 700987 words/s, in_qsize 5, out_qsize 2
2018-02-05 13:51:01,203 : INFO : EPOCH 45 - PROGRESS: at 90.16% examples, 702557 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:51:02,211 : INFO : EPOCH 45 - PROGRESS: at 95.05% examples, 703117 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:51:03,171 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-02-05 13:51:03,179 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-05 13:51:03,191 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-02-05 13:51:03,207 : INFO : worker thr

2018-02-05 13:51:57,426 : INFO : EPOCH 48 - PROGRESS: at 66.53% examples, 760331 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:51:58,428 : INFO : EPOCH 48 - PROGRESS: at 71.77% examples, 761411 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:51:59,433 : INFO : EPOCH 48 - PROGRESS: at 77.03% examples, 762069 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:52:00,435 : INFO : EPOCH 48 - PROGRESS: at 82.37% examples, 763345 words/s, in_qsize 6, out_qsize 1
2018-02-05 13:52:01,438 : INFO : EPOCH 48 - PROGRESS: at 88.07% examples, 767412 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:52:02,448 : INFO : EPOCH 48 - PROGRESS: at 93.62% examples, 769853 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:52:03,453 : INFO : EPOCH 48 - PROGRESS: at 99.35% examples, 773359 words/s, in_qsize 7, out_qsize 0
2018-02-05 13:52:03,545 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-02-05 13:52:03,548 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-02-05 13:52:03,

50 epoch model trained and saved.


In [9]:
# test w2v model
model.wv.most_similar('asshole')

2018-02-05 13:52:44,905 : INFO : precomputing L2-norms of word weight vectors


[('idiot', 0.8594728708267212),
 ('arsehole', 0.7313122749328613),
 ('loser', 0.7033094167709351),
 ('douchebag', 0.7018705606460571),
 ('retard', 0.6967161297798157),
 ('jerk', 0.6913732290267944),
 ('as', 0.6844013929367065),
 ('imbecile', 0.6830329895019531),
 ('moron', 0.6682279109954834),
 ('fucking', 0.6651079654693604)]